In [1]:
from datasets import load_dataset

In [5]:
fw_sample = load_dataset("HuggingFaceFW/fineweb-edu", name="sample-10BT", cache_dir="data/raw/", num_proc=12)

Resolving data files:   0%|          | 0/1630 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9672101 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/104 [00:00<?, ?it/s]

In [7]:
fw_sample

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'dump', 'url', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score'],
        num_rows: 9672101
    })
})

In [17]:
fw_filtered = fw_sample["train"].filter(lambda x: x["score"] >= 3.69, num_proc=14)
fw_filtered.num_rows

Filter (num_proc=14):   0%|          | 0/9672101 [00:00<?, ? examples/s]

682847

In [18]:
fw_filtered

Dataset({
    features: ['text', 'id', 'dump', 'url', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score'],
    num_rows: 682847
})

In [19]:
train_test = fw_filtered.train_test_split(test_size=.1, seed=42)
train_set = train_test["train"]
val_set = train_test["test"]

In [23]:
train_set.save_to_disk("../data/train_fineweb_edu_369")
val_set.save_to_disk("../data/val_fineweb_edu_369")


Saving the dataset (0/7 shards):   0%|          | 0/614562 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/68285 [00:00<?, ? examples/s]